# Preprocessing for IRFs calculation

This notebook can be run only by LST collaboration members and is provided only for internal reproductibility

## Imports and setup

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import lstmcpipe
print("lstmcpipe", lstmcpipe.__version__)

import lstchain
print("lstchain", lstchain.__version__)

import pyirf
print("pyirf", pyirf.__version__)

In [ ]:
import multiprocessing
from importlib import reload
from pathlib import Path
import pickle
import astropy.units as u
from copy import deepcopy
from astropy.table import vstack
from lstchain.io import read_mc_dl2_to_QTable

import utils
import dl2_to_irfs

In [ ]:
import matplotlib.pyplot as plt
def get_color_cycle(num_colors):
    cm = plt.get_cmap('tab20c')
    return [cm(1.*i/num_colors) for i in range(num_colors)]

In [ ]:
from astropy.io.ascii import write, read

# select pointings

In [ ]:
pointings_table = read('../../2_mc_simulations/2.3_mc_pointings/pointings_test.ecsv')
pointings_table

-------
## load data

In [ ]:
indir = '/fefs/aswg/data/mc/DL2/AllSky/20221027_v0.9.9_crab_tuned/TestingDataset/dec_2276'

In [ ]:
filenames = []
for p in pointings_table:
    filenames.append(list(Path(indir, p['dirname']).glob('dl2_*.h5'))[0])
    
pointings_table['dl2_filename'] = filenames

In [ ]:
pointings_table

In [ ]:
events_dict = {}
for filename in filenames:
    utils.load_event_dict(filename, events_dict)

### Merging per altitude, with symmetrical azimuths from the magnetic North

In [ ]:
stacked_alt = dl2_to_irfs.merge_per_alt(events_dict, pointings_table)

In [ ]:
stacked_alt[46.803]['simu_info']

In [ ]:
stacked_pickle_filemame = 'stacked_alt_source_indep.pickle'

In [ ]:
if not Path(stacked_pickle_filemame).exists():
    with open(stacked_pickle_filemame, 'wb') as file:
        pickle.dump(stacked_alt, file)
else:
    raise FileExistsError("overwrite?")

In [ ]:
Path(stacked_pickle_filemame).exists(), f"{Path(stacked_pickle_filemame).lstat().st_size/1024**3:.2f}GB"

In [ ]:
with open(stacked_pickle_filemame, 'rb') as file:
    stacked_alt = pickle.load(file)

In [ ]:
stacked_alt.keys()

## Parameters

A global theta cut is applied to all events to remove very bad events, especially the ones at low energies that have been assigned the wrong reconstructed direction.

A more restrictive theta cut is applied to compute the energy resolution and bias.

**Resolutions are not computed if there are less than 100 events in the bin**

In [ ]:
# efficiencies = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
efficiencies = [0.4, 0.7, 0.9]
theta_containment=0.7

erange = (15*u.GeV, 25*u.TeV)


In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
colors

In [ ]:
output_dir = 'irfs'
Path(output_dir).mkdir(exist_ok=True)

In [ ]:
processes = []

for alt in stacked_alt.keys():
    for efficiency in efficiencies:
        print(alt, efficiency)
        outfile = Path(output_dir, f'irfs_zen_{90-alt:.2f}_gh-eff_{efficiency}.fits.gz')
        p = multiprocessing.Process(target=dl2_to_irfs.produce_irfs_pyirf,
                                    args = (stacked_alt[alt]['params'],
                                            stacked_alt[alt]['simu_info'],
                                            efficiency,
                                            outfile,
                                           ),
                                    kwargs= dict(
                                            theta_containment=theta_containment,
                                            srcdep=False,)
                                   )
        p.start()
        processes.append(p)

for p in processes:
    p.join()



In [ ]:
ls irfs_fine_bins/

----

## Source dependent IRFs

Source dependent analysis has been done with MC in a limited range of offsets.    
A limit at 1 deg was finally chosen.

In [ ]:
off = 1
indir = Path(f'/fefs/aswg/workspace/seiya.nozaki/Crab_performance_paper/20221027_v0.9.9_crab_tuned/combined_off_axis_{off}deg/DL2_MC/data')

list(indir.iterdir())[:2]

In [ ]:
filenames_srcdep = [list(Path(indir).glob(f"*{p['dirname']}*"))[0] for p in pointings_table]
pointings_table['srcdep_filename'] = filenames_srcdep
pointings_table

In [ ]:
events_dict_srcdep = {}
for p in pointings_table:
    node = p['dirname']
    print(node)
    # filename = [filename for filename in filenames_srcdep if node in filename.as_posix()][0]
    if node not in events_dict_srcdep:
        filename =  p['srcdep_filename']
        events_dict_srcdep[node] = {'path': filename}
        events_dict_srcdep[node]['params'], events_dict_srcdep[node]['simu_info'] = read_mc_dl2_to_QTable(filename)
        # Patch for the coma aberration bias
        utils.patch_events_altaz(events_dict_srcdep[node]['params'])

In [ ]:
grp = utils.groupby_alt(pointings_table)
grp

In [ ]:
stacked_alt_srcdep = {}
for alt, index in grp.items():
    print(alt)
    stacked_alt_srcdep[alt]={}
    stacked_alt_srcdep[alt]['params'] = vstack([events_dict_srcdep[p['dirname']]['params'] for p in pointings_table[list(index)]])
    stacked_alt_srcdep[alt]['simu_info'] = deepcopy(events_dict_srcdep[pointings_table[list(index)[0]]['dirname']]['simu_info'])
    stacked_alt_srcdep[alt]['simu_info'].n_showers = sum([events_dict_srcdep[p['dirname']]['simu_info'].n_showers for p in pointings_table[list(index)]])

In [ ]:
stacked_srcdep_pickle_filemame = 'stacked_alt_src_dep.pickle'

In [ ]:
if not Path(stacked_srcdep_pickle_filemame).exists():
    with open(stacked_srcdep_pickle_filemame, 'wb') as file:
        pickle.dump(stacked_alt_srcdep, file)
else:
    raise FileExistsError("overwrite?")
    
Path(stacked_srcdep_pickle_filemame).exists(), f"{Path(stacked_srcdep_pickle_filemame).lstat().st_size/1024**3:.2f}GB"  

In [ ]:
with open(stacked_srcdep_pickle_filemame, 'rb') as file:
    stacked_alt_srcdep = pickle.load(file)

In [ ]:
output_dir_srcdep = 'irfs_srcdep'
Path(output_dir_srcdep).mkdir(exist_ok=True)

In [ ]:
processes = []

for alt in stacked_alt_srcdep.keys():
    for efficiency in efficiencies:
        print(alt, efficiency)
        outfile = Path(output_dir_srcdep, f'irfs_zen_{90-alt:.2f}_gh-eff_{efficiency}_srcdep.fits.gz')
        p = multiprocessing.Process(target=dl2_to_irfs.produce_irfs_pyirf,
                                    args=(stacked_alt_srcdep[alt]['params'], 
                                           stacked_alt_srcdep[alt]['simu_info'], 
                                           efficiency, 
                                           outfile),
                                    kwargs=dict( alpha_containment=0.7,
                                       srcdep=True,
                                               )
                                   )
        p.start()
        processes.append(p)

for p in processes:
    p.join()
                            
